# vector swap



In [1]:
from generate import *
from closest_vs_minimal import *

# def gram_matrix(matrix) -> matrix:
#     return matrix * matrix.transpose()

# def vector_to_list(vector) -> list:
#     return [float(num) for num in vector.list()]

def find_real_minimum_genfile(G, current_row, inserted_component) -> vector:
    """
    returns linear combination of the minimum over real numbers
    the >current_row< parameter is the coordinate of fixed col/row idk
    the >K< parameter is the number inserted into this column
    """
    G = matrix_to_list(G)
    dimension=len(G)
    # print(dimension, "dimension in FRM")
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    # matrixA[0,0] = 1
    # print("G",matrix(G))
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row][j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row][j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    # print("A", matrixA,"\n","b", -(1)*matrixB, )
    result = vector_to_list(matrixA.solve_right((-1)*(inserted_component) * matrixB))
    result.insert(current_row-1, inserted_component)
    return vector(result)

In [2]:
def swapping(basis_as_matrix):
    basis_as_list= matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    # print("B = ",basis_as_list)
    used_vectors =[[0]*dim]
    used_vectors.extend(basis_as_list)

    G = gram_matrix(basis_as_matrix)

    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    # print("Basis na začatku: ")
    # for i in range(dim):
    #         print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
    # print("--------------")
    basis_as_matrix=matrix(basis_as_list)
    # print(basis_as_matrix)
    # print(basis_as_list)

    i = 0
    while i < dim:
#     for j in range(0,dimension):
        # print("\n")
        # print(i)
        real_min = vector_to_list(find_real_minimum_genfile(G, i%dim, 1))

        # print(evaluate_norms_at_cube(cube_points(real_min), basis_as_list))
        
        shortestLC = shortest_lc_in_cube(real_min, basis_as_matrix)
        # print(real_min)
        # print(shortestLC)
        shortest_vector = vector(shortestLC)*basis_as_matrix
        # print("sv", shortest_vector,(shortest_vector).norm().n())
        # print("used vectors",used_vectors)
        if vector_to_list(shortest_vector) in used_vectors:
              break
            #   print("ERROR: I already used the vector ", shortest_vector)
              i+=1
              continue
        if shortest_vector.norm() > basis_as_matrix[-1].norm():
              break
            #   print("ERROR, LONGER VECTOR")
              i+=1
              continue
        # print(type(shortest_vector))
        shortest_vector = vector_to_list(shortest_vector)
        used_vectors.append(shortest_vector)
        # print("basis to be thrown away", basis_as_list[dimension-1])
        basis_as_list[dim-1] = shortest_vector
        # print(basis_as_list)
        # print("basis new, should be SV", basis_as_list[dimension-1])
        basis_as_list.sort(key=lambda x: euclidean_norm(x))
        # print(basis_as_list)
        # for i in range(dim):
                # print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
        basis_as_matrix = matrix(basis_as_list)
        G = gram_matrix(basis_as_matrix)
    # print("reduced lll",  (matrix_to_list(matrix(ZZ, basis_as_list).LLL())), euclidean_norm(first_nonzero_norm_vector(matrix_to_list(matrix(ZZ, basis_as_list).LLL()))))
    return euclidean_norm(basis_as_list[0])


    
# možná se ještě vyplatí opravit cube takto: co když sii jako real-min uložím něco, co ale je moc daleko a  
# fixací JINÉ souřadnice ale dostanu už správnej výsledek? ještě na to mrkni viko
# tot je na opravu funkce compare with cube

In [3]:
def gaussian_reduction(b_1, b_2):
    #  = basis_as_matrix[0]
    # b_2 = basis_as_matrix[1]
    b_1 = vector(b_1)
    b_2 = vector(b_2)
    if b_1.norm() < b_2.norm():
        b_1 , b_2 = b_2, b_1
    r = round(b_1.dot_product(b_2) / b_2.norm()**2)
    a = b_1 - r * b_2
    if a.norm() < b_2.norm():
        a, b_2 = gaussian_reduction(a, b_2)
    return a, b_2

# gaussian_reduction(Matrix([[1,2],[5,7]]))
# gaussian_reduction(Matrix([[11,4],[19,8]]))
gaussian_reduction([11,4,4],[19,8,1])

((8, 4, -3), (-3, 0, -7))

In [50]:
from sage.modules.free_module_integer import IntegerLattice
from generate import *

def vector_to_int_list(vector) -> list:
    return [int(num) for num in vector.list()]

def semaev(basis_as_matrix):
    # print(basis_as_matrix)
    basis_as_list= matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    iterace = 0
    shortest_vectors_sequence = []
    while True:
        shortest_vectors_sequence.append(basis_as_matrix[0].norm().n())
        iterace += 1
        # Reduce the pair b_1 and b_2
        basis_as_list[1], basis_as_list[0] = gaussian_reduction(basis_as_list[1], basis_as_list[0])
        basis_as_list[1] = vector_to_int_list(basis_as_list[1])
        basis_as_list[0] = vector_to_int_list(basis_as_list[0])

        basis_as_matrix=matrix(ZZ,basis_as_list)

        # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis_as_matrix)
        lc_cube = vector_to_list(find_real_minimum_mistni(G, dim-1, 1))
        slc = shortest_lc_in_cube(lc_cube, basis_as_matrix)
        # print("slc", slc)
        a = vector(ZZ,slc) * basis_as_matrix
        # if I found a shorter vector a, continue
        if a.norm().n() < basis_as_matrix[dim-1].norm().n() and vector_to_int_list(a) != [0]*dim and vector_to_int_list(a) not in basis_as_list:
            basis_as_list[dim-1] = (a)
            basis_as_list.sort(key=lambda x: euclidean_norm(x))
            # for basevector in basis_as_list:
            #     print(euclidean_norm(basevector))
            # print("-")
            basis_as_matrix=matrix(ZZ,basis_as_list)
            # print(basis_as_matrix[0].norm().n())
            continue
        break
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    return basis_as_matrix, iterace

def find_real_minimum_mistni(G, current_row, inserted_component) -> vector:
    dimension = len(matrix_to_list(G))
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right(-(inserted_component) * matrixB)).list()
    result.insert(current_row, inserted_component)
    return vector(result).n()



exp je  6
Range čísel v prvním sloupci: deset na  6 -> average počet iterací 17.6
exp je  10
Range čísel v prvním sloupci: deset na  10 -> average počet iterací 24.8
exp je  14
Range čísel v prvním sloupci: deset na  14 -> average počet iterací 44.4
exp je  18
Range čísel v prvním sloupci: deset na  18 -> average počet iterací 50.0
exp je  22
Range čísel v prvním sloupci: deset na  22 -> average počet iterací 59.5
exp je  26
Range čísel v prvním sloupci: deset na  26 -> average počet iterací 64.6
exp je  30


KeyboardInterrupt: 

In [ ]:
# TADY POČÍTÁM POČET ITERACÍ VERSUS VELIKOST ČÍSEL MATII 



for exp in range(6,40,4):
    avg = 0
    iterations = 10
    # prime = random_prime(10^exp, proof=True)
    print("exp je ", exp)
    for j in range(iterations):
        # avg += semaev(hard_instance_lattice(n=4, r=1, q=prime))[1]
        avg += semaev(random_special_matrix(6, 10^exp))[1]
        # print()
    print("Range čísel v prvním sloupci: deset na ", exp, "-> average počet iterací", float(avg/iterations))

# semaev(random_special_matrix(3, 10^10,))
# semaev(matrix([[-73, -50, -63], [60, 59, -53], [92, 83, 27]]))
# print(random_special_matrix(d, r))



In [28]:
from hardinstancelattice import hard_instance_lattice

#semaev(hard_instance_lattice(n=4, r=1, q=113))

slc [-7, -2, 1]
slc [2, -6, 1]
slc [0, 0, 1]
poměry
19.6001700672894
1.00000000000000


(
[ 0  1 -2  1]   
[ 1 -1  0  3]   
[-6  5  3  3], 3
)

In [71]:
# ZDE POČÍTÁM ÚSPĚŠNOST SEMAIEVA


from math import isclose

def counter(dimension : int, perimeter, tries, function):
    res = {"True":0, "False":0}
    for i in range(tries):
        # mojematice = random_special_matrix(dimension, perimeter)
        mojematice = randomMatrix(dimension, perimeter)
        swapresult = function(mojematice)[0]
        semaev_shortest = euclidean_norm(matrix_to_list(swapresult)[0])
        L = IntegerLattice(mojematice, lll_reduce=False)
        sv_exact = L.shortest_vector()
        sv_exact_norm = sv_exact.norm().n()
        lllnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(mojematice.LLL())))

        if  isclose(semaev_shortest, sv_exact_norm, abs_tol=1e-5):
            res["True"]+=1
            continue
        # print(matrix_to_list(mojematice))
        print("falsuje", semaev_shortest,"exact: ", sv_exact_norm, "LLLko je ", lllnorm)
        res["False"]+=1
    return res

# print(counter(3, 1000000, 10, semaev))

# print(counter(4, 1000000, 100, semaev))


# print(counter(6, 1000000, 100, semaev))


# print(counter(10, 1000000, 10, semaev))


print(counter(4, 1000000, 1000, semaev_modified_for_higher_bases))
# print(counter(6, 1000000, 100, semaev_modified_for_higher_bases))
# print(counter(10, 1000000, 10, semaev_modified_for_higher_bases))

# for i in range(10):
#     semaeov_basis = semaev_modified_for_higher_bases(randomMatrix(10, 1000))
#     print(semaeov_basis[0].norm().n(),"\n")

# konstantni_sada_matic_6 = [matrix_to_list(randomMatrix(6, 1000)) for i in range(100)]
# konstantni_sada_matic_4 = [matrix_to_list(randomMatrix(4, 1000)) for i in range(100)]
# konstantni_sada_matic_10 = [matrix_to_list(randomMatrix(4, 1000)) for i in range(10)]


{'True': 1000, 'False': 0}


In [ ]:
def semaev_modified_for_higher_bases(basis_as_matrix):
    # print(basis_as_matrix)
    basis_as_list = matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    iterace = 0
    L = IntegerLattice(basis_as_matrix, lll_reduce=False)
    sv_exact = L.shortest_vector()
    sv_exact_norm = sv_exact.norm().n()
    # print(sv_exact_norm, "exact norm")
    while True:
        iterace += 1
        # print(basis_as_matrix, "\n")
        # Reduce the pair b_1 and b_2
        basis_as_list[1], basis_as_list[0] = gaussian_reduction(basis_as_list[1], basis_as_list[0])
        basis_as_list[1] = vector_to_int_list(basis_as_list[1])
        basis_as_list[0] = vector_to_int_list(basis_as_list[0])

        basis_as_matrix=matrix(ZZ,basis_as_list)

        # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis_as_matrix)
        lc_cube = vector_to_list(find_real_minimum_mistni(G, dim - 1, 1))
        slc = shortest_lc_in_cube(lc_cube, basis_as_matrix)
        a = vector(ZZ, slc) * basis_as_matrix

        # if I found a shorter vector a, continue
        if a.norm().n() < basis_as_matrix[dim-1].norm().n() and vector_to_int_list(a) != [0]*dim and vector_to_int_list(a) not in basis_as_list:
            basis_as_list[dim-1] = vector_to_int_list(a)
            basis_as_list.sort(key=lambda x: euclidean_norm(x))
            basis_as_matrix=matrix(ZZ,basis_as_list)
            # print("continue", a.norm().n() , basis_as_matrix[dim-1].norm().n())
            continue
        elif a.norm().n() >= basis_as_matrix[dim-1].norm().n(): # pokud to už nezkratim 
            # print("rekurzuju, aktualni sv ma normu ", basis_as_matrix[0].norm().n())
            basis_as_matrix[0:3] = matrix(semaev(matrix(basis_as_list[0:3]))[0])
        # else:
            # print(a.norm().n() < basis_as_matrix[2].norm().n() , vector_to_int_list(a) != [0]*dim , vector_to_int_list(a) not in basis_as_list)
        break
    # print(iterace, "iterace")
    return basis_as_matrix, iterace




In [54]:
def first_nonzero_norm_vector(vectors):
    for v in vectors:
        if euclidean_norm(v) != 0:
            return v
        
# def testik(matice):
#     matice = matrix(matice)
#     lllnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(matrix(ZZ, matice).LLL())))
#     return swapping(matice) == lllnorm

# # B_1 = [[-8, -9, 7], [10, 9, -6], [-8, 4, 10]]
# # assert testik(B_1)

# # B_2 = [[9, -1, -6], [7, -9, -5], [9, 6, -2]]
# # assert testik(B_2) # NEJEDE

# B_3 = [[3, 10, 4], [-4, 2, 7], [-4, -8, -3]]
# assert testik(B_3) # FUNGUJE

# B_4 =  [[6, -4, -5], [9, 6, -10], [2, 9, 2]]
# assert testik(B_4) # FUNGUJE
